# Embedded ML Lab - Challenge (testing yolo example)

This is an example of inference with the VOC data set and tinyyolov2. There are pretrained weights (`voc_pretrained.pt`) stored that can be loaded into the model.

In [6]:
%load_ext autoreload
%autoreload 2
import torch

from faf.utils.dataloader import VOCDataLoader
loader = VOCDataLoader(train=False, batch_size=1, path="./data")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from faf.tinyyolov2 import TinyYoloV2
from faf.utils.yolo import nms, filter_boxes
from faf.utils.viz import display_result

# make an instance with 20 classes as output
net = TinyYoloV2(num_classes=20)

# load pretrained weights
sd = torch.load("./weights/voc_pretrained.pt")
net.load_state_dict(sd)

#put network in evaluation mode
net.eval()

TinyYoloV2(
  (pad): ReflectionPad2d((0, 1, 0, 1))
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv6): Conv2d(256, 512, kernel_size=(3, 3), stride=

In [8]:
import faf.person_only as fpo
import importlib
importlib.reload(fpo)
net.state_dict().keys()

val = next(iter(loader))

new_net = fpo.strip_classes(net)
print(new_net(val[0]).shape)
print(net(val[0]).shape)

torch.Size([1, 5, 10, 10, 6])
torch.Size([1, 5, 10, 10, 25])


In [ ]:
import tqdm
for idx, (input, target) in tqdm.tqdm(enumerate(loader), total=len(loader)):
    
    #input is a 1 x 3 x 320 x 320 image
    output = net(input)
    "output is of a tensor of size 32 x 125 x 10 x 10"
    #output is a 32 x 125 x 10 x 10 tensor
    
    #filter boxes based on confidence score (class_score*confidence)
    output = filter_boxes(output, 0.1)
    
    #filter boxes based on overlap
    output = nms(output, 0.25)
    
    display_result(input, output, target, file_path='yolo_prediction.png')

    if idx > 10:
        break